## Docker 安装 ES & Kibana 8.12.2版本

```
docker network create elastic-net
```








In [2]:
!pip install elasticsearch==8.12.1

## 建立ES连接

In [5]:
# Import Elasticsearch package
from elasticsearch import Elasticsearch

# Connect to the elastic cluster
es = Elasticsearch("http://localhost:9200")
resp = es.info()
print(resp)

{'name': 'edf62d159d66', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'gr-cG0S0Reuj4Hj8gI2ouA', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


## 创建索引

In [7]:
# 索引名称
index_name = "my_index"

# 定义索引的映射（Mapping）
mapping = {
    "mappings": {
        "properties": {
            "name": {"type": "text"},
            "age": {"type": "integer"},
            "about": {"type": "text"},
            "interests": {"type": "keyword"}
        }
    }
}

# 使用关键字参数创建索引
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)
    print(f"索引 {index_name} 创建成功。")
else:
    print(f"索引 {index_name} 已经存在。")

索引 my_index 创建成功。


## 添加数据

In [8]:
# 索引文档（即插入数据）
document1 = {
    "name": "John Doe",
    "age": 42,
    "about": "John Doe is a nice guy who lives in New York.",
    "interests": ["sports", "music"]
}

document2 = {
    "name": "Jane Smith",
    "age": 36,
    "about": "Jane Smith loves to go rock climbing.",
    "interests": ["rock climbing", "martial arts"]
}

# 将文档添加到索引中
response1 = es.index(index=index_name, id=1, body=document1)
response2 = es.index(index=index_name, id=2, body=document2)

print(f"文档1索引成功: {response1['result']}")
print(f"文档2索引成功: {response2['result']}")

文档1索引成功: created
文档2索引成功: created


## 检索数据

In [9]:
# 搜索文档
search_body = {
    "query": {
        "match": {
            "name": "John Doe"
        }
    }
}

# 执行搜索
search_results = es.search(index=index_name, body=search_body)
print("搜索结果：")
for hit in search_results['hits']['hits']:
    print(hit["_source"])


搜索结果：
{'name': 'John Doe', 'age': 42, 'about': 'John Doe is a nice guy who lives in New York.', 'interests': ['sports', 'music']}


## 删除数据

In [10]:
# 删除文档
delete_response = es.delete(index=index_name, id=1)
print(f"文档删除: {delete_response['result']}")

文档删除: deleted


## 删除索引

In [13]:
# 使用关键字参数删除索引
if es.indices.exists(index=index_name):
    response = es.indices.delete(index=index_name)
    print(f"索引 {index_name} 删除成功。响应: {response}")
else:
    print(f"索引 {index_name} 不存在，无法删除。")

索引 my_index 删除成功。响应: {'acknowledged': True}
